In [1]:
import numpy as np, pandas as pd
from Bio import SeqIO

In [2]:
#refpath = '/home/croth/Hypermutator_mac/DATA/FungiDB-46_CneoformansH99_Genome.fasta'
refpath = '/home/croth/Desktop/SSK1/FungiDB-43_CneoformansH99_Genome.fasta'

REF = [s for s in SeqIO.parse(refpath,format='fasta')]

In [3]:
## Set paths to genotype dataframes
gt_path = '../GENOTYPE/Bt65xH99_F1_progeny-SNPS.csv.gz'

In [4]:
## Load in genotype data
## Genotypes per site
SNP = pd.read_csv(gt_path,index_col=0)

In [5]:
clens = pd.DataFrame([(s.id,len(s.seq)) for s in REF],columns=['Seqid','Length'])
clens['Contig'] = -1
clens['Chrom'] = -1


chrom_remap = pd.DataFrame([(c,int(c.split("_")[-1])) 
                            for c in  sorted(SNP.Contig.unique())],
                           columns = ['Contig','Chrom'])

for i,j in chrom_remap.iterrows():
    clens.loc[j.Chrom-1,'Contig'] = j.Contig
    clens.loc[j.Chrom-1,'Chrom'] = int(j.Chrom)
    
clens.replace(-1,'M',inplace=True)

clens['Cumsum'] = [0] + list(clens.Length.cumsum()[:-1])
clens['Midpts'] = clens.Length/2 + clens.Cumsum

## Calculate the number of SNPs per chromosome
clens['Nsnps'] = [SNP[(SNP.Contig==c)].shape[0] for c in clens.Contig]

## View tail
clens.tail()

,Seqid,Length,Contig,Chrom,Cumsum,Midpts,Nsnps
10,CP003830.1,1561994,Chr_11,11,14855526,15636523.0,18069
11,CP003831.1,774062,Chr_12,12,16417520,16804551.0,9015
12,CP003832.1,756744,Chr_13,13,17191582,17569954.0,8509
13,CP003833.2,942867,Chr_14,14,17948326,18419759.5,11172
14,CP003834.1,24919,M,M,18891193,18903652.5,0


In [6]:
## SAve out
clens.to_csv('../GENOTYPE/H99_chrommap.csv.gz',index=False)